<a href="https://colab.research.google.com/github/gibranfp/CursoAprendizajeAutomatizado/blob/master/notebooks/1a_naive_bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clasificador bayesiano ingenuo: ejemplo

Curso: [Aprendizaje Automatizado](http://turing.iimas.unam.mx/~gibranfp/cursos/aprendizaje_automatizado). Profesor: [Gibran Fuentes Pineda](https://turing.iimas.unam.mx/~gibranfp/). Ayudantes: [Bere](https://turing.iimas.unam.mx/~bereml/) y [Ricardo](https://turing.iimas.unam.mx/~ricardoml/) Montalvo Lezama.

---
---

## ¿Deportes o Informática?

Tenemos un conjunto de entrenamiento de 11 documentos que pertenecen a las clases: Deportes (0) o Informática (1). Nuestro Objetivo en esta libreta es construir un clasificador Bayesiano Ingenuo usando el modelo multinomial para clasificar documentos nuevos. 

In [1]:
import os

import numpy as np
import pandas as pd

In [2]:
DATA_DIR = "../data"
filename = "dep_inf.csv"
filepath = os.path.join(DATA_DIR, filename)
URL = "https://raw.githubusercontent.com/gibranfp/CursoAprendizajeAutomatizado/master/data/dep_inf.csv"

---
### 1 Carga de datos

 Cada documento esta formado por un vector de 8 dimensiones donde cada dimensión representa la frecuencia de las palabras en nuestro vocabulario de interés:
 
 
$$V =
      \begin{vmatrix}
      w_1 = gol & w_2 = computación & w_3 = transmitir & w_4= velocidad\\
      w_5 = \textit{técnica} & w_6= defensa & w_7=\textit{desempeño} & w_8=campo \\
      \end{vmatrix}  
 $$
 
 Lectura con Pandas:

In [3]:
! mkdir {DATA_DIR}
! wget -nc {URL} -O {filepath}
! echo {filepath}
df = pd.read_csv(filepath)
df

mkdir: cannot create directory '../data': File exists
File '../data/dep_inf.csv' already there; not retrieving.
../data/dep_inf.csv


,w1,w2,w3,w4,w5,w6,w7,w8,c
0,2,0,0,0,1,2,3,1,0
1,0,0,1,0,2,1,0,0,0
2,0,1,0,1,0,2,1,0,0
3,1,0,0,2,0,1,0,1,0
4,2,0,0,0,1,0,1,3,0
5,0,0,1,2,0,0,2,1,0
6,0,1,1,0,0,0,1,0,1
7,1,2,0,1,0,0,1,1,1
8,0,1,1,0,0,2,0,0,1
9,0,0,0,0,0,0,0,0,1


### 2 Estimación de parámetros

Conversion a numpy:

In [4]:
data = df.to_numpy()

Separación de clases:

In [5]:
data_d = data[data[:, -1] == 0]
x_d = data_d[:, :-1]
x_d

array([[2, 0, 0, 0, 1, 2, 3, 1],
       [0, 0, 1, 0, 2, 1, 0, 0],
       [0, 1, 0, 1, 0, 2, 1, 0],
       [1, 0, 0, 2, 0, 1, 0, 1],
       [2, 0, 0, 0, 1, 0, 1, 3],
       [0, 0, 1, 2, 0, 0, 2, 1]])

In [6]:
data_i = data[data[:, -1] == 1]
x_i = data_i[:, :-1]
x_i

array([[0, 1, 1, 0, 0, 0, 1, 0],
       [1, 2, 0, 1, 0, 0, 1, 1],
       [0, 1, 1, 0, 0, 2, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 1, 0, 1, 0]])

### Estimacion de a priori

Asumimos que la clases se distribuyen como una Bernoulli quedando expresadas por:

$$ P(C) = q^C(1-q)^{1-C} $$

donde la estimación del parámetro se reduce al conteo de frecuencia de la clase en los datos:

$$ \hat{q}_C = \frac{N_C}{N}$$

Implementando para la clase Deportes el estimado $q_s$ es:

In [7]:
q_d = x_d.shape[0] / data.shape[0]
q_d

0.5454545454545454

Para la clase Informática el estimado $q_i$ es:

In [8]:
q_i = 1 - q_d
q_i

0.4545454545454546

### Estimación de verosimilitudes

Asumimos que la atributos de distribuyen como una multinomial:

$$ P(x|C) = \frac{n !}{\prod_{t=1}^{|V|} x_{t}!} \prod_{t=1}^{|V|} q(w_{t}|C)^{x_t} $$

Sin embargo, podemos ignorar el termino de normalización:

$$ P(x|C) \propto \prod_{t=1}^{|V|} q(w_{t}|C)^{x_t} $$

Es estimado de cada parámetro es la frecuencia de la palabra en la clase:

$$\hat{q}(w_{t}|C) = \frac{n_C(w_t)}{\sum_{s=1}^{|V|} n_C(w_t)}$$

Contemos primero el numero de ocurrencias de las palabras en las clases $n_C(w_t)$:

In [9]:
n_d_w = x_d.sum(axis=0) 
n_d_w

array([5, 1, 2, 5, 4, 6, 7, 6])

In [10]:
n_i_w = x_i.sum(axis=0) 
n_i_w

array([1, 4, 3, 1, 1, 2, 3, 1])

Estimemos los parámetros $\hat{q}(w|C)$ de la distribución Multinomial:

In [11]:
p_w_d = n_d_w / n_d_w.sum()
p_w_d

array([0.13888889, 0.02777778, 0.05555556, 0.13888889, 0.11111111,
       0.16666667, 0.19444444, 0.16666667])

In [12]:
p_w_i = n_i_w / n_i_w.sum()
p_w_i

array([0.0625, 0.25  , 0.1875, 0.0625, 0.0625, 0.125 , 0.1875, 0.0625])

### 3. Clasificación de nuevos documentos

Recordemos que para clasificar en el modelo bayseiano ingenuo debemos computar:

$$ P'(N|S) = \prod_{t=1}^{|V|} q(w_{t}|S)^{x_t} P (S)$$
$$ P'(N|I) = \prod_{t=1}^{|V|} q(w_{t}|I)^{x_t} P (I)$$

$$C = \operatorname*{max} \bigg\{P(S) P'(N | S), P(I) P'(N | I) \bigg\} $$

Clasificando el documento $N_1$:

In [13]:
N1 = [2, 1, 0, 0, 1, 2, 0, 1]

Clase deportes $P(N_1 | D)$:

In [14]:
p_n1_d = q_d * np.power(p_w_d, N1).prod()
p_n1_d

1.5034701534472906e-07

Clase informatica $P(N_1 | I)$:

In [15]:
p_n1_i = q_i * np.power(p_w_i, N1).prod()
p_n1_i

2.7093020352450286e-08

Clase más probable:

In [16]:
CLASSES = ['DEPORTES', 'INFORMATICA']
C = np.argmax([p_n1_d, p_n1_i])
CLASSES[C]

'DEPORTES'

Clasifiquemos el documento $N_2$:

In [17]:
N2 = [0, 1, 1, 0, 1, 0, 1, 0]

Clase deportes $P(N_2 | D)$:

In [18]:
p_n2_d = q_d * np.power(p_w_d, N2).prod()
p_n2_d

1.818597497609843e-05

Clase deportes $P(N_2 | I)$:

In [19]:
p_n2_i = q_i * np.power(p_w_i, N2).prod()
p_n2_i

0.00024968927556818187

Clase más probable:

In [20]:
C = np.argmax([p_n2_d, p_n2_i])
CLASSES[C]

'INFORMATICA'